In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

# Uncomment the below to use LangSmith. Not required.
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
import accelerate
import bitsandbytes
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

c:\Users\saksh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
!huggingface-cli login --token 

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\saksh\.cache\huggingface\token
Login successful


In [12]:
%pip install -q langchain_fireworks

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os 
os.environ['FIREWORKS_API_KEY'] = ''

In [3]:
from langchain_fireworks import ChatFireworks
llm = ChatFireworks(model="accounts/fireworks/models/firefunction-v1")

In [4]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [22]:
documents

[Document(id_='1164', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software which enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nAI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., c

In [24]:
documents[0].text

'Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software which enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nAI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, oft

In [5]:
text = """
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software which enable machines to perceive their environment and uses learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
AI technology is widely used throughout industry, government, and science. Some high-profile applications include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once something becomes useful enough and common enough it's not labeled AI anymore."
Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence. Artificial intelligence was founded as an academic discipline in 1956. The field went through multiple cycles of optimism, followed by periods of disappointment and loss of funding, known as AI winter. Funding and interest vastly increased after 2012 when deep learning surpassed all previous AI techniques, and after 2017 with the transformer architecture. This led to the AI boom of the early 2020s, with companies, universities, and laboratories overwhelmingly based in the United States pioneering significant advances in artificial intelligence.
"""


In [8]:
from langchain_core.documents import Document

documents = [Document(page_content=text)]

In [ ]:
import requests

link = "https://en.wikipedia.org/wiki/Machine_learning"
f = requests.get(link)
print(f.text)


In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
%pip install neo4j


In [9]:
from langchain_community.document_loaders import WikipediaLoader

query = "Hayao Miyazaki"
raw_documents = WikipediaLoader(query=query).load()

In [10]:
# create doc_text as list to store text
doc_text = [d.page_content for d in raw_documents]

In [11]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
import os
# load Diffbot Token for later entity and relationship extraction
diffbot_api_key = ""
os.environ["DIFFBOT_API_KEY"] = diffbot_api_key
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

# use Diffbot Graph Transformer to extract entities/relationships from Wikipedia articles
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

In [1]:
from langchain_community.graphs import Neo4jGraph

url = ""  # Adjust as necessary
username = ""
password = ""

graph = Neo4jGraph(url=url, username=username, password=password)

In [13]:
graph.add_graph_documents(graph_documents)

In [17]:
from neo4j import GraphDatabase

# Assuming 'llm_transformer.convert_to_graph_documents(documents)' is correctly returning the objects.

# graph_documents = llm_transformer.convert_to_graph_documents(documents)

# print(f"Nodes:{graph_documents[0].nodes}")
# print(f"Relationships:{graph_documents[0].relationships}")

# Establish a connection to the Neo4j database

uri = ""  # Adjust as necessary
username = ""
password = ""

driver = GraphDatabase.driver(url, auth=(username, password))

def add_node(tx, node_id, node_type):
    query = "MERGE (n:Node {id: $id, type: $type}) RETURN n"
    tx.run(query, id=node_id, type=node_type)

def add_relationship(tx, source_id, target_id, relationship_type):
    query = (
        "MATCH (a:Node {id: $source_id}), (b:Node {id: $target_id}) "
        "MERGE (a)-[r:RELATIONSHIP {type: $relationship_type}]->(b)"
    )
    tx.run(query, source_id=source_id, target_id=target_id, relationship_type=relationship_type)

nodes = graph_documents[0].nodes
relationships = graph_documents[0].relationships

# Execute the operations
with driver.session() as session:
    # Add nodes
    for node in nodes:
        session.execute_write(add_node, node.id, node.type)
    
    # Add relationships
    for relationship in relationships:
        session.execute_write(add_relationship, relationship.source.id, relationship.target.id, relationship.type)

driver.close()


In [ ]:
%pip install langchain-cohere


In [ ]:
%pip install tokenizers==0.14

In [ ]:
%pip install huggingface-hub==0.19.3

In [40]:
%pip install upgrade quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-gemini 0.1.7 requires google-generativeai<0.5.0,>=0.4.1, but you have google-generativeai 0.5.2 which is incompatible.


In [41]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("")

In [ ]:
%pip install llamaapi

In [16]:
from langchain_openai import OpenAIEmbeddings
import os
from llama_index.core import Settings

from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage

openaiapi=
os.environ["OPENAI_API_KEY"] = 
os.environ["ANTHROPIC_API_KEY"] = 
# embd = OpenAIEmbeddings()
from langchain_cohere import CohereEmbeddings

embd = CohereEmbeddings(cohere_api_key=)


from llamaapi import LlamaAPI

# Replace 'Your_API_Token' with your actual API token
llama = LlamaAPI()

from langchain_experimental.llms import ChatLlamaAPI

model = ChatLlamaAPI(client=llama)


In [19]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture

RANDOM_SEED = 224  # Fixed seed for reproducibility

### - Code from citations referenced above (added comments and docstrings) - ###


def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]


def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters


### - Our code below - ###


def embed(texts):
    """
    Generate embeddings for a list of text documents.

    This function assumes the existence of an `embd` object with a method `embed_documents`
    that takes a list of texts and returns their embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be embedded.

    Returns:
    - numpy.ndarray: An array of embeddings for the given text documents.
    """
    text_embeddings = embd.embed_documents(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np


def embed_cluster_texts(texts):
    """
    Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

    This function combines embedding generation and clustering into a single step. It assumes the existence
    of a previously defined `perform_clustering` function that performs clustering on the embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be processed.

    Returns:
    - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
    """
    text_embeddings_np = embed(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    return df


def fmt_txt(df: pd.DataFrame) -> str:
    """
    Formats the text documents in a DataFrame into a single string.

    Parameters:
    - df: DataFrame containing the 'text' column with text documents to format.

    Returns:
    - A single string where all text documents are joined by a specific delimiter.
    """
    unique_txt = df["text"].tolist()
    return "- - \n - - ".join(unique_txt)


def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
    clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
    the content within each cluster.

    Parameters:
    - texts: A list of text documents to be processed.
    - level: An integer parameter that could define the depth or detail of processing.

    Returns:
    - Tuple containing two DataFrames:
      1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
      2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
         and the cluster identifiers.
    """

    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)

    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []

    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)

    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()

    print(f"Generated {len(all_clusters)} clusters")

    # Summarization
    template = """Here is a sub-set of LangChain Expression Langauge doc. 
    
    LangChain Expression Langauge provides a way to compose chain in LangChain.
    
    Give a detailed summary of the documentation provided.
    
    Documentation:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model 
    # | StrOutputParser()

    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Recursively embeds, clusters, and summarizes texts up to a specified level or until
    the number of unique clusters becomes 1, storing the results at each level.

    Parameters:
    - texts: List[str], texts to be processed.
    - level: int, current recursion level (starts at 1).
    - n_levels: int, maximum depth of recursion.

    Returns:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
      levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
    """
    results = {}  # Dictionary to store results at each level

    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # Store the results of the current level
    results[level] = (df_clusters, df_summary)

    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        if type(new_texts)!=type("f"):
            return
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)

    return results

In [27]:
uri = ""  # Adjust as necessary
username = ""
password = ""

print(graph.schema)

Node properties:
Person {id: STRING, name: STRING, positionHeld: STRING, dateOfBirth: STRING, causeOfDeath: STRING, dateOfDeath: STRING}
Organization {id: STRING, name: STRING, foundingDate: STRING}
Award {id: STRING, name: STRING}
Location {id: STRING, name: STRING}
Skill {id: STRING, name: STRING}
Relationship properties:
EMPLOYEE_OR_MEMBER_OF {evidence: STRING, startTime: STRING, positionHeld: STRING, isNotCurrent: STRING, isCurrent: STRING}
AWARDS {evidence: STRING}
PLACE_OF_BIRTH {evidence: STRING}
PERSON_LOCATION {evidence: STRING}
INTERESTED_IN {evidence: STRING}
INDUSTRY {evidence: STRING}
NATIONALITY {evidence: STRING}
WORK_RELATIONSHIP {evidence: STRING}
FOUNDED_BY {evidence: STRING}
FAMILY_MEMBER {evidence: STRING, isCurrent: STRING}
SOCIAL_RELATIONSHIP {evidence: STRING, isCurrent: STRING}
SIBLING {evidence: STRING}
HAS_PARENT {evidence: STRING}
HAS_CHILD {evidence: STRING}
AUTHOR_OF {evidence: STRING, pointInTime: STRING}
ORGANIZATION_LOCATIONS {evidence: STRING, isCurrent

In [33]:
from langchain_community.vectorstores import Chroma

# Initialize all_texts with leaf_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)

# Now, use all_texts to build the vectorstore with Chroma
vectorstore = Chroma.from_texts(texts=all_texts, embedding=embd)
retriever = vectorstore.as_retriever()

In [32]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

# Prompt
prompt = hub.pull("rlm/rag-prompt")


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Question
originalAns_achievement = rag_chain.invoke("Tell me about Hayao Miyazaki's career achievements.")

In [35]:
originalAns_achievement

'Hayao Miyazaki is a renowned Japanese animator, filmmaker, and manga artist. He co-founded Studio 
      Ghibli and has directed many critically acclaimed and commercially successful animated films, 
      including Spirited Away which won the Academy Award for Best Animated Feature in 2003.
      Miyazaki is widely regarded as one of the most accomplished filmmakers in the history of animation.'


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
os.environ["GOOGLE_API_KEY"] = ""

# Initialize a Fireworks chat model
chat = ChatGoogleGenerativeAI(model="gemini-pro")

In [4]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=chat,
    qa_llm=chat,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True
)

In [5]:
chain = GraphCypherQAChain.from_llm(
    chat, graph=graph, verbose=True, return_intermediate_steps=True
)

In [9]:
result_kg_achievements = chain("Tell me about Hayao Miyazaki's career awards")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Hayao Miyazaki"})-[:AWARDS]->(a:Award) RETURN a.name
Full Context:
[{'a.name': 'Person of Cultural Merit'}, {'a.name': 'Academy Award for Best Animated Feature'}, {'a.name': 'Academy Honorary Award'}, {'a.name': 'Japan Academy Prize for Picture of the Year'}]

> Finished chain.


In [11]:
result_kg_achievements

{'query': "Tell me about Hayao Miyazaki's career awards",
 'result': 'Hayao Miyazaki has won a Person of Cultural Merit, an Academy Award for Best Animated Feature, an Academy Honorary Award, and a Japan Academy Prize for Picture of the Year.',
 'intermediate_steps': [{'query': 'MATCH (p:Person {name: "Hayao Miyazaki"})-[:AWARDS]->(a:Award) RETURN a.name'},
  {'context': [{'a.name': 'Person of Cultural Merit'},
    {'a.name': 'Academy Award for Best Animated Feature'},
    {'a.name': 'Academy Honorary Award'},
    {'a.name': 'Japan Academy Prize for Picture of the Year'}]}]}

In [10]:
result_kg_colleagues = chain("Who work with Hayao Miyazaki?")
result_kg_colleagues



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person {name: "Hayao Miyazaki"})-[r:EMPLOYEE_OR_MEMBER_OF]->(o:Organization)
MATCH (p2:Person)-[r2:EMPLOYEE_OR_MEMBER_OF]->(o)
WHERE p1 <> p2
RETURN p2.name
Full Context:
[{'p2.name': 'Mamoru Hosoda'}, {'p2.name': 'Daikichiro Kusube'}, {'p2.name': 'Isao Takahata'}, {'p2.name': 'Toshio Suzuki'}, {'p2.name': 'Seijun Suzuki'}, {'p2.name': 'Gorô Miyazaki'}, {'p2.name': 'Isao Takahata'}, {'p2.name': 'Isao Takahata'}]

> Finished chain.


{'query': 'Who work with Hayao Miyazaki?',
 'result': 'Mamoru Hosoda, Daikichiro Kusube, Toshio Suzuki, Seijun Suzuki, Gorô Miyazaki, and Isao Takahata work with Hayao Miyazaki.',
 'intermediate_steps': [{'query': 'MATCH (p1:Person {name: "Hayao Miyazaki"})-[r:EMPLOYEE_OR_MEMBER_OF]->(o:Organization)\nMATCH (p2:Person)-[r2:EMPLOYEE_OR_MEMBER_OF]->(o)\nWHERE p1 <> p2\nRETURN p2.name'},
  {'context': [{'p2.name': 'Mamoru Hosoda'},
    {'p2.name': 'Daikichiro Kusube'},
    {'p2.name': 'Isao Takahata'},
    {'p2.name': 'Toshio Suzuki'},
    {'p2.name': 'Seijun Suzuki'},
    {'p2.name': 'Gorô Miyazaki'},
    {'p2.name': 'Isao Takahata'},
    {'p2.name': 'Isao Takahata'}]}]}

In [12]:
originalAns_colleagues = rag_chain.invoke("Who work with Hayao Miyazaki?")
originalAns_colleagues

According to the context, Hayao Miyazaki co-founded Studio Ghibli with 
Isao Takahata in 1985. Miyazaki and Takahata had a long 
history of collaboration, working together at Toei Animation and other 
studios on various projects before founding Studio Ghibli. 

In [ ]:
with open('nodes.txt','w') as f:
    f.write(str(graph_documents[0].nodes))

In [14]:
with open('relationships.txt','w') as f:
    f.write(str(graph_documents[0].relationships))